<a href="https://colab.research.google.com/github/spdr-lily/Data-Science-Profile/blob/main/chatbot_com_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=998433b47e7c4c9aa292294a4906d0300c8402919f9802093d1a9ed433a73f24
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
import nltk
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import random
import wikipedia

In [ ]:
# Baixar recursos do NLTK
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Configurar Wikipedia em português
wikipedia.set_lang("pt")

In [ ]:
# Carregar modelo de análise de sentimentos baseado em Transformers
sentiment_pipeline = pipeline(model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

Device set to use cpu


In [ ]:
def analyze_sentiment(user_input): # nalisa o sentimento da mensagem usando TextBlob, NLTK VADER e um modelo baseado em Transformers.
    textblob_polarity = TextBlob(user_input).sentiment.polarity
    vader_polarity = sia.polarity_scores(user_input)['compound']
    transformer_sentiment = sentiment_pipeline(user_input)[0]['score']

    # Ajustando o sinal da pontuação do Transformer para manter a consistência
    if sentiment_pipeline(user_input)[0]['label'] == 'negative':
        transformer_sentiment = -transformer_sentiment

    # Média das análises
    final_polarity = (textblob_polarity + vader_polarity + transformer_sentiment) / 3
    return final_polarity

In [ ]:
def chatbot_response(user_input): # gerar uma resposta baseada na análise de sentimento com mais variações e limiares ajustados.
    sentiment = analyze_sentiment(user_input)

    positive_responses = [
        "Isso é ótimo! Como posso ajudar ainda mais?",
        "Fico feliz em saber disso! O que mais posso fazer por você?",
        "Que notícia maravilhosa! Conte-me mais."
    ]

    neutral_responses = [
        "Entendi. Você pode me contar mais?",
        "Interessante! Quer compartilhar mais detalhes?",
        "Ok! Como posso ajudar?"
    ]

    slightly_negative_responses = [
        "Parece que algo está te incomodando. Quer me contar mais?",
        "Sinto que há algo errado. Como posso ajudar?",
        "Se precisar desabafar, estou aqui para ouvir."
    ]

    negative_responses = [
        "Sinto muito por isso. Como posso ajudar a melhorar seu dia?",
        "Parece que você não está tendo um bom dia. Quer conversar mais sobre isso?",
        "Lamento ouvir isso. Estou aqui para ajudar, se precisar."
    ]

    if sentiment > 0.5:
        return random.choice(positive_responses)
    elif sentiment > 0.1:
        return random.choice(neutral_responses)
    elif sentiment > -0.3:
        return random.choice(slightly_negative_responses)
    else:
        return random.choice(negative_responses)

In [ ]:
def fetch_wikipedia_info(query):
    """Busca informações da Wikipedia sobre o tema perguntado."""
    try:
        resultados = wikipedia.search(query)
        if not resultados:
            return "Desculpe, não encontrei informações sobre esse assunto."
        melhor_resultado = resultados[0]
        texto = wikipedia.page(melhor_resultado)
        return f"{texto.summary}\n\nFonte: {melhor_resultado} - Wikipedia"
    except Exception as e:
        return f"Houve um erro ao buscar informações: {str(e)}"

In [ ]:
def main():
    print("Chatbot: Olá! Como posso te ajudar hoje?")
    while True:
        user_input = input("Você: ")
        if user_input.lower() in ["sair", "exit", "tchau"]:
            print("Chatbot: Até mais! Tenha um ótimo dia.")
            break

        wiki_response = fetch_wikipedia_info(user_input)
        print("Chatbot:", wiki_response)

        feedback = input("O que achou da resposta?\n")
        sentimento = sentiment_pipeline(feedback)

        if sentimento[0]['label'] == "positive" and sentimento[0]['score'] > 0.6:
            print("Chatbot: Fico feliz que tenha gostado da resposta!")
        elif sentimento[0]['label'] == "negative" and sentimento[0]['score'] > 0.6:
            print("Chatbot: Que pena que não gostou da resposta. Gostaria de tentar uma outra pergunta?")
        else:
            print("Chatbot: Obrigado pelo feedback, volte sempre.")

In [ ]:
if __name__ == "__main__":
    main()

Chatbot: Olá! Como posso te ajudar hoje?
Você: O que é PLN?
Chatbot: Processamento de língua natural (PLN) é uma subárea da ciência da computação, inteligência artificial e da linguística que estuda os problemas da geração e compreensão automática de línguas humanas naturais. Sistemas de geração de língua natural convertem informação de bancos de dados de computadores em linguagem compreensível ao ser humano e sistemas de compreensão de língua natural convertem ocorrências de linguagem humana em representações mais formais, mais facilmente manipuláveis por programas de computador. Alguns desafios do PLN são compreensão de língua natural, fazer com que computadores extraiam sentido de linguagem humana ou natural e geração de língua natural.

Fonte: Processamento de linguagem natural - Wikipedia
O que achou da resposta?
Eu gostei muito!
Chatbot: Fico feliz que tenha gostado da resposta!
Você: tchau
Chatbot: Até mais! Tenha um ótimo dia.
